In [1]:
# 将数据转化为向量
# 本demo的数据都是txt文件，打开的话是一堆01字符串 比如下面这个
# trainingDigits/0_0.txt文件：
# 00000000000001111000000000000000
# 00000000000011111110000000000000
# 00000000001111111111000000000000
# 00000001111111111111100000000000
# 00000001111111011111100000000000
# 00000011111110000011110000000000
# 00000011111110000000111000000000
# 00000011111110000000111100000000
# 00000011111110000000011100000000
# 00000011111110000000011100000000
# 00000011111100000000011110000000
# 00000011111100000000001110000000
# 00000011111100000000001110000000
# 00000001111110000000000111000000
# 00000001111110000000000111000000
# 00000001111110000000000111000000
# 00000001111110000000000111000000
# 00000011111110000000001111000000
# 00000011110110000000001111000000
# 00000011110000000000011110000000
# 00000001111000000000001111000000
# 00000001111000000000011111000000
# 00000001111000000000111110000000
# 00000001111000000001111100000000
# 00000000111000000111111000000000
# 00000000111100011111110000000000
# 00000000111111111111110000000000
# 00000000011111111111110000000000
# 00000000011111111111100000000000
# 00000000001111111110000000000000
# 00000000000111110000000000000000
# 00000000000011000000000000000000
# 可以看出来是个手写的0对吧？

In [2]:
# 偷懒一下 用上次的kNN实现
import operator
import numpy as np
# 实现一下kNN 
def classify0(inX, dataSet, labels, k):
    #numpy函数shape[0]返回dataSet的行数
    dataSetSize = dataSet.shape[0]
    #在列向量方向上重复inX共1次(横向),行向量方向上重复inX共dataSetSize次(纵向)
    diffMat = np.tile(inX, (dataSetSize, 1)) - dataSet
    #二维特征相减后平方
    sqDiffMat = diffMat**2
    #sum()所有元素相加,sum(0)列相加,sum(1)行相加
    sqDistances = sqDiffMat.sum(axis=1)
    #开方,计算出距离
    distances = sqDistances**0.5
    #返回distances中元素从小到大排序后的索引值
    sortedDistIndices = distances.argsort()
    #定一个记录类别次数的字典
    classCount = {}
    for i in range(k):
        #取出前k个元素的类别
        voteIlabel = labels[sortedDistIndices[i]]
        #dict.get(key,default=None),字典的get()方法,返回指定键的值,如果值不在字典中返回默认值。
        #计算类别次数
        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1
    #python3中用items()替换python2中的iteritems()
    #key=operator.itemgetter(1)根据字典的值进行排序
    #key=operator.itemgetter(0)根据字典的键进行排序
    #reverse降序排序字典
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    print(sortedClassCount)
    #返回次数最多的类别,即所要分类的类别
    return sortedClassCount[0][0]

In [3]:
# 将txt内容转换成便于处理的行向量
'''
 这里打开文件的时候需要注意
 一开始我用的是 file = open(filename)
 但是后面读入的向量为全0……直接导致我后面训练出的错误率高达82% 人都傻了
 后来逐步排查 发现导入数据的时候就出了问题（因为我用sklearn来测试也是82%，所以我判断不是算法的问题，只可能是数据）
 于是就改成了这种文件读取方式
'''
def img2vector(filename):
    # 创建一个长度为1024的行向量 每个字母都是 32x32 = 1024 的
    returnVect = np.zeros((1,1024))
     # @ 此处更正
    with open(filename) as file:
        for i in range(32):
            # 逐行读入
            strline = file.readline()
            for j in range(32):
                returnVect[0, 32*i+j]=int(strline[j])
            
        return returnVect

In [4]:
arr = img2vector('trainingDigits/0_1.txt')
print(arr[0,:32])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
# 手写识别测试
from os import listdir
def handwritingClassTest():
    hwLabels = []
    trainingFileList = listdir('trainingDigits') # 需要导入os库的listdir 帮助我们获取训练集的文件目录 进行训练文件的读取，方便训练
    m = len(trainingFileList)
    trainingMat = np.zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0] # 取出第i个文件的文件名并且去掉后缀
        classNumStr = int(fileStr.split('_')[0]) # 取出文件名中的数字 文件名格式为"x_x.txt" 第一个x代表的数字就是文件内手写体的值，也即ground truth
        hwLabels.append(classNumStr)
        trainingMat[i,:] = img2vector('trainingDigits/%s' % fileNameStr) # 将该数字对应的向量存入矩阵
    # 测试数据
    testFileList = listdir('testDigits') # 测试数据同理
    errorCount = 0.0
    lenTest = len(testFileList)
    for i in range(lenTest):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        vectorUnderTest = img2vector('testDigits/%s' % fileNameStr)
        classificationRes = classify0(vectorUnderTest, trainingMat, hwLabels, 3)
        print("The classifier came back with: %d, the real answer is : %d" % (classificationRes, classNumStr) )
        if (classificationRes != classNumStr):
            errorCount+=1.0
    print("\nThe total number of errors is:%d" % errorCount)
    print("\nThe error rate is:%f" % (errorCount/float(lenTest)))


In [10]:
# 用sklearn的库来整一个？
from sklearn.neighbors import KNeighborsClassifier as kNN
def handwritingClassTest2():
    #测试集的Labels
    hwLabels = []
    #返回trainingDigits目录下的文件名
    trainingFileList = listdir('trainingDigits')
    #返回文件夹下文件的个数
    m = len(trainingFileList)
    #初始化训练的Mat矩阵,测试集
    trainingMat = np.zeros((m, 1024))
    #从文件名中解析出训练集的类别
    for i in range(m):
        #获得文件的名字
        fileNameStr = trainingFileList[i]
        #获得分类的数字
        classNumber = int(fileNameStr.split('_')[0])
        #将获得的类别添加到hwLabels中
        hwLabels.append(classNumber)
        #将每一个文件的1x1024数据存储到trainingMat矩阵中
        trainingMat[i,:] = img2vector('trainingDigits/%s' % (fileNameStr))
    #构建kNN分类器
    neigh = kNN(n_neighbors = 3, algorithm = 'auto')
    #拟合模型, trainingMat为训练矩阵,hwLabels为对应的标签
    neigh.fit(trainingMat, hwLabels)
    #返回testDigits目录下的文件列表
    testFileList = listdir('testDigits')
    #错误检测计数
    errorCount = 0.0
    #测试数据的数量
    mTest = len(testFileList)
    #从文件中解析出测试集的类别并进行分类测试
    for i in range(mTest):
        #获得文件的名字
        fileNameStr = testFileList[i]
        #获得分类的数字
        classNumber = int(fileNameStr.split('_')[0])
        #获得测试集的1x1024向量,用于训练
        vectorUnderTest = img2vector('testDigits/%s' % (fileNameStr))
        #获得预测结果
        # classifierResult = classify0(vectorUnderTest, trainingMat, hwLabels, 3)
        classifierResult = neigh.predict(vectorUnderTest)
        print("分类返回结果为%d\t真实结果为%d" % (classifierResult, classNumber))
        if(classifierResult != classNumber):
            errorCount += 1.0
    print("总共错了%d个数据\n错误率为%f%%" % (errorCount, errorCount/mTest * 100))

In [12]:
# 测试！
if __name__ == "__main__":
    # handwritingClassTest() # 自己实现的kNN
    handwritingClassTest2() #sklearn的kNN

分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0	真实结果为0
分类返回结果为0

分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为5	真实结果为5
分类返回结果为6	真实结果为6
分类返回结果为6	真实结果为6
分类返回结果为6	真实结果为6
分类返回结果为6	真实结果为6
分类返回结果为6	真实结果为6
分类返回结果为6	真实结果为6
分类返回结果为6	真实结果为6
分类返回结果为6	真实结果为6
分类返回结果为6	真实结果为6
分类返回结果为6

In [8]:
# 测试出来的错误率均为1%左右